In [13]:
from database.db_config import current_db as db

In [14]:
# Alter the headers types, before querying.
db.execute_ddl('ALTER TABLE "All continents updated" ALTER COLUMN "TotalCases" TYPE INT USING "TotalCases"::integer,ALTER COLUMN "NewCases" TYPE INT, ALTER COLUMN "TotalDeaths" TYPE INT USING "TotalDeaths"::integer,ALTER COLUMN "NewDeaths" TYPE INT, ALTER COLUMN "TotalRecovered" TYPE INT USING "TotalRecovered"::integer,ALTER COLUMN "NewRecovered" TYPE INT, ALTER COLUMN "ActiveCases" TYPE INT USING "ActiveCases"::integer,ALTER COLUMN "SeriousCritical" TYPE INT USING "SeriousCritical"::integer, ALTER COLUMN "scrap_date" SET DATA TYPE date USING to_date(cast("scrap_date" as TEXT), \'yyyy-mm-dd\')')
db.execute_ddl('ALTER TABLE "All countries updated" ALTER COLUMN "TotalCases" TYPE INT USING "TotalCases"::integer,ALTER COLUMN "NewCases" TYPE INT, ALTER COLUMN "TotalDeaths" TYPE INT USING "TotalDeaths"::integer,ALTER COLUMN "NewDeaths" TYPE INT, ALTER COLUMN "TotalRecovered" TYPE INT USING "TotalRecovered"::integer,ALTER COLUMN "NewRecovered" TYPE INT, ALTER COLUMN "ActiveCases" TYPE INT USING "ActiveCases"::integer,ALTER COLUMN "SeriousCritical" TYPE INT USING "SeriousCritical"::integer,ALTER COLUMN "scrap_date" SET DATA TYPE date USING to_date(cast("scrap_date" as TEXT), \'yyyy-mm-dd\')')


DDL executed successfully.
DDL executed successfully.


In [15]:
countries_minMax = db.sql_query('SELECT "Country", "TotalCases" FROM "All countries updated" '
                              'WHERE "TotalCases" IN ((SELECT MAX("TotalCases") FROM "All countries updated"),'
                              '(SELECT MIN("TotalCases") FROM "All countries updated"))')
print(countries_minMax)

+---------+------------+
| Country | TotalCases |
+---------+------------+
|   USA   |  15113903  |
| Vanuatu |     1      |
+---------+------------+


In [16]:
# TotalCases Min and Max by a continent.
continents_minMax = db.sql_query('SELECT "Continent", "TotalCases" FROM "All continents updated" '
                              'WHERE "TotalCases" IN ((SELECT MAX("TotalCases") FROM "All continents updated" WHERE "Continent" <>  \'World\'),'
                              '(SELECT MIN("TotalCases") FROM "All continents updated")) ')
print(continents_minMax)

+-----------+------------+
| Continent | TotalCases |
+-----------+------------+
|   Europe  |  18439062  |
|  Oceania  |   45927    |
+-----------+------------+


In [17]:
# Top 15 countries with the highest number of cases.
top15_totalCases = db.sql_query('SELECT "Country","Continent","TotalCases",RANK() OVER(PARTITION BY "Continent" ORDER BY "TotalCases" DESC) AS "Continent_rank_by_TotalCases","ActiveCases","SeriousCritical" FROM (SELECT "Country_id","Continent_id" FROM "All Countries") AS t1 JOIN  (SELECT * FROM "All Continents") AS t2 USING("Continent_id") JOIN  (SELECT "Country","Country_id","TotalCases","ActiveCases","SeriousCritical"  FROM "All countries updated" LIMIT 15) AS t3 USING("Country_id")')
print(top15_totalCases)

+-----------+---------------+------------+------------------------------+-------------+-----------------+
|  Country  |   Continent   | TotalCases | Continent_rank_by_TotalCases | ActiveCases | SeriousCritical |
+-----------+---------------+------------+------------------------------+-------------+-----------------+
|   India   |      Asia     |  9676801   |              1               |    398040   |       8944      |
|    Iran   |      Asia     |  1040547   |              2               |    259439   |       5809      |
|   Russia  |     Europe    |  2460770   |              1               |    479891   |       2300      |
|   France  |     Europe    |  2292497   |              2               |   2067756   |       3220      |
|   Italy   |     Europe    |  1728878   |              3               |    755306   |       3454      |
|     UK    |     Europe    |  1723242   |              4               |     None    |       1274      |
|   Spain   |     Europe    |  1699145   |    

In [18]:
# Countries that are located in Oceania continent.
oceania_countries = db.sql_query('SELECT "Country" FROM "All Countries" WHERE "Continent_id" IN (SELECT '
                                 '"Continent_id" FROM "All Continents" WHERE "Continent" = \'Oceania\')')
print(oceania_countries)

+------------------+
|     Country      |
+------------------+
|    Australia     |
|   New Zealand    |
| French Polynesia |
| Papua New Guinea |
|       Fiji       |
|  New Caledonia   |
+------------------+


In [19]:
# The count of countries in each continent.
countries_perContinent = db.sql_query('SELECT "Continent","Countires_count" FROM (SELECT * FROM "All Continents") AS ''t1 JOIN (SELECT "Continent_id", COUNT("Country") AS "Countires_count" FROM "All Countries" GROUP BY "Continent_id") AS t2 USING("Continent_id") ORDER BY "Countires_count" DESC')
print(countries_perContinent)

+---------------+-----------------+
|   Continent   | Countires_count |
+---------------+-----------------+
|     Africa    |        57       |
|      Asia     |        49       |
|     Europe    |        48       |
| North America |        39       |
| South America |        14       |
|    Oceania    |        6        |
+---------------+-----------------+


In [20]:
# The country with the highest "TotalCases" in each continent.
country_high_perContinent = db.sql_query('SELECT "Country","Continent","TotalCases" FROM (SELECT "Country","Continent","TotalCases" ,RANK() OVER(PARTITION BY "Continent" ORDER BY "TotalCases" DESC) AS "rank" FROM (SELECT "Country","Country_id","TotalCases" FROM "All countries updated" ) AS t1  JOIN (SELECT "Country_id", "Continent_id" FROM "All Countries") AS t2 USING("Country_id")  JOIN (SELECT "Continent_id","Continent" FROM "All Continents") AS t3 USING("Continent_id")) AS t4 WHERE "rank" =1;')
print(country_high_perContinent)

+--------------+---------------+------------+
|   Country    |   Continent   | TotalCases |
+--------------+---------------+------------+
| South Africa |     Africa    |   814565   |
|    India     |      Asia     |  9676801   |
|    Russia    |     Europe    |  2460770   |
|     USA      | North America |  15113903  |
|  Australia   |    Oceania    |   27965    |
|    Brazil    | South America |  6582606   |
+--------------+---------------+------------+


In [21]:
# Top 5 countries with active cases.
top5_countries_active = db.sql_query('SELECT "Country","ActiveCases" FROM "All countries updated" WHERE "ActiveCases" '
                                   'IS NOT NULL ORDER BY "ActiveCases" DESC LIMIT 5;')
print(top5_countries_active)

+---------+-------------+
| Country | ActiveCases |
+---------+-------------+
|   USA   |   5988387   |
|  France |   2067756   |
|  Italy  |    755306   |
|  Brazil |    644522   |
| Belgium |    533503   |
+---------+-------------+


In [22]:
# The date that israel had the highest value of new cases.
israel_newCases_date = db.sql_query('SELECT "scrap_date" AS "max_NewCases_date" FROM "Israel" WHERE "NewCases" = (Select MAX("NewCases") FROM "Israel")')
print(israel_newCases_date)

+-------------------+
| max_NewCases_date |
+-------------------+
|     2020-09-27    |
+-------------------+


In [23]:
# Ranking each continent by its active cases.
continent_active_rank = db.sql_query('SELECT "Continent","ActiveCases", RANK() OVER(ORDER BY "ActiveCases" DESC) FROM "All continents updated" WHERE "Continent" <> \'World\'')
print(continent_active_rank)

+---------------+-------------+------+
|   Continent   | ActiveCases | rank |
+---------------+-------------+------+
|     Europe    |   9801896   |  1   |
| North America |   6451450   |  2   |
|      Asia     |   1739002   |  3   |
| South America |    946026   |  4   |
|     Africa    |    285249   |  5   |
|    Oceania    |    11753    |  6   |
+---------------+-------------+------+


In [24]:
israel_minMax_dates = db.sql_query('SELECT MIN("scrap_date") as "Earliest Date",MAX("scrap_date") as "Latest Date" FROM "Israel"')
print(israel_minMax_dates)

+---------------+-------------+
| Earliest Date | Latest Date |
+---------------+-------------+
|   2020-08-02  |  2020-12-06 |
+---------------+-------------+
